# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [1]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [2]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [3]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [4]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,1,36,,72,,2018-02-22 12:42:42 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,15,17,,79,,2018-02-22 12:41:52 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,20,7,,82,,2018-02-22 12:45:06 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
3,,30,30,,83,,2018-02-22 12:42:43 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,9,10,,119,,2018-02-22 12:43:19 PM,40.696089,,-73.978034,,Park Ave & St Edwards St,,Park Ave & St Edwards St,1,In Service,False,19


In [5]:
# there is one row for each bike station.  How many stations are there? 
len(df)

813

In [6]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

array([-74.0969366 ,  40.65539977, -73.9077436 ,  40.81439444])

In [7]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

array([  957370.14732176,   178056.37414829,  1009803.53178432,
         235986.42166716])

In [8]:
boros

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330466.075042,1.623827e+09,(POLYGON ((970217.0223999023 145643.3322143555...
1,3,Brooklyn,741185.900596,1.937597e+09,(POLYGON ((1021176.479003906 151374.7969970703...
2,4,Queens,897040.298576,3.045168e+09,(POLYGON ((1029606.076599121 156073.8142089844...
3,1,Manhattan,358408.460709,6.364467e+08,(POLYGON ((981219.0557861328 188655.3157958984...
4,2,Bronx,464400.198868,1.186973e+09,(POLYGON ((1012821.805786133 229228.2645874023...


In [9]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

414 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [10]:
manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

414 stations in Manhattan


In [11]:
bronx = boros.geometry[4]
in_mn = df.geometry.within(bronx)
print(sum(in_mn), 'stations in bronx')

0 stations in bronx


In [12]:
queens = boros.geometry[2]
in_mn = df.geometry.within(queens)
print(sum(in_mn), 'stations in queens')

81 stations in queens


In [13]:
brooklyn = boros.geometry[1]
in_mn = df.geometry.within(brooklyn)
print(sum(in_mn), 'stations in brooklyn')

266 stations in brooklyn


In [14]:
staten_island = boros.geometry[0]
in_mn = df.geometry.within(staten_island)
print(sum(in_mn), 'stations in staten island')

0 stations in staten island


In [15]:
staten_island = boros.geometry[0]
in_mn = df.geometry.within(staten_island)
print(sum(df['availableBikes'][in_mn]))

0


In [16]:
brooklyn = boros.geometry[1]
in_mn = df.geometry.within(brooklyn)
print(sum(df['availableBikes'][in_mn]))

2373


In [ ]:
brooklyn = boros.geometry[1]
in_mn = df.geometry.within(brooklyn)
print(sum(df['availableBikes'][in_mn]))